# Extract and analyse summary features

The clustered and projected data can be used to extract different features:

- size per cluster
- mean intensity per cluster
- TODO add more

The features are saved as an adata object.

In [1]:
from miann.tl import Experiment, FeatureExtractor
import os
from miann.pl import plot_mean_intensity, zscore, get_intensity_change, plot_intensity_change, plot_mean_size, plot_size_change
import anndata as ad
from miann.utils import init_logging
init_logging()

## Extract features
Features can be extracted with `python extract_features.py test/CondVAE_pert-CC clustering_0.5 intensity`


In [2]:
exp = Experiment.from_dir('test/CondVAE_pert-CC')
# just use one dir here, for extracting all features, run the script
data_dir = exp.data_params['data_dirs'][0]
extr = FeatureExtractor(exp, data_dir=data_dir, cluster_name='clustering_0.5', 
                        cluster_dir='aggregated/sub-0.005')

INFO:Experiment:Setting up experiment test/CondVAE_pert-CC
INFO:Experiment:Initialised from existing experiment in test/CondVAE_pert-CC


In [7]:
obj_id = extr.mpp_data.unique_obj_ids[0]
sub_mpp_data = extr.mpp_data.subset(obj_ids=[obj_id], copy=True)

INFO:MPPData:Before subsetting: 557 objects
INFO:MPPData:Subsetting to 1 objects
INFO:MPPData:Created new: MPPData for NascentRNA (17008 mpps with shape (1, 1, 34) from 1 objects). Data keys: ['x', 'y', 'obj_ids', 'mpp', 'labels', 'clustering_0.5', 'latent', 'conditions'].


In [15]:
adata = sub_mpp_data.get_adata()

In [18]:
adata.obsm['spatial']

array([[1412.,  112.],
       [1413.,  112.],
       [1414.,  112.],
       ...,
       [1377.,  257.],
       [1378.,  257.],
       [1379.,  257.]], dtype=float32)

In [32]:
sub_mpp_data.get_adata(obs=[extr.params['cluster_name']])

AnnData object with n_obs × n_vars = 17008 × 34
    obs: 'mapobject_id', 'plate_name', 'well_name', 'well_pos_y', 'well_pos_x', 'tpoint', 'zplane', 'label', 'is_border', 'mapobject_id_cell', 'plate_name_cell', 'well_name_cell', 'well_pos_y_cell', 'well_pos_x_cell', 'tpoint_cell', 'zplane_cell', 'label_cell', 'is_border_cell', 'is_mitotic', 'is_mitotic_labels', 'is_polynuclei_HeLa', 'is_polynuclei_HeLa_labels', 'is_polynuclei_184A1', 'is_polynuclei_184A1_labels', 'is_SBF2_Sphase_labels', 'is_SBF2_Sphase', 'Heatmap-48', 'cell_cycle', 'description', 'dimensions', 'id', 'cell_type', 'EU', 'duration', 'perturbation', 'secondary_only', 'siRNA', 'perturbation_duration', 'LocalDensity_Nuclei_800', 'TR', 'TR_factor', 'TR_norm', 'MERGE_KEY', 'clustering_0.5'
    var: 'name'
    obsm: 'spatial'

In [76]:
# TODO continue here with computing co-occ scores

In [75]:
import numpy as np
import squidpy as sq

#interval = np.logspace(np.log2(args.co_minval),np.log2(args.co_maxval),args.co_nsteps, base=2).astype(np.float32)
#interval = np.linspace(args.co_minval,args.co_maxval,args.co_nsteps).astype(np.float32)
               

def extract_co_occurrence(self, interval):
    """FeatureExtractor
    Extract co_occurrence for each cell invididually. 

    Adds obsm co_occurrence_CLUSTER1_CLUSTER2 to adata and saves to self.fname

    Args:
        interval: distance intervals for which to calculate co-occurrence score
    """
    self.log.info(f"calculating co-occurrence for intervals {interval} and clustering {self.params['cluster_name']} (col: {self.params['cluster_col']})")
    cluster_names = {n: i for i,n in enumerate(self.clusters)}
    obj_ids = []
    co_occs = []
    for obj_id in self.mpp_data.unique_obj_ids[:10]:
        adata = self.mpp_data.subset(obj_ids=[obj_id], copy=True).get_adata(obs=[self.params['cluster_name']])
        # ensure that cluster annotation is present in adata
        if self.params['cluster_name'] != self.params['cluster_col']:
            adata.obs[self.params['cluster_col']] = annotate_clustering(adata.obs[self.params['cluster_name']], self.annotation, 
                self.params['cluster_name'], self.params['cluster_col'])
        adata.obs[self.params['cluster_col']] = adata.obs[self.params['cluster_col']].astype('category')
        self.log.info(f'co-occurrence for {obj_id}, with shape {adata.shape}')
        cur_co_occ, cur_interval = sq.gr.co_occurrence(
            adata,
            cluster_key=self.params['cluster_col'],
            spatial_key='spatial',
            interval=interval,
            copy=True, show_progress_bar=False,
        )
        # ensure that co_occ has correct format incase of missing clusters
        co_occ = np.zeros((len(self.clusters),len(self.clusters),len(interval)-1))
        cur_clusters = np.vectorize(cluster_names.__getitem__)(np.array(adata.obs[self.params['cluster_col']].cat.categories))
        grid = np.meshgrid(cur_clusters, cur_clusters)
        co_occ[grid[0].flat, grid[1].flat] = cur_co_occ.reshape(-1, len(interval)-1)
        co_occs.append(co_occ.copy())
        obj_ids.append(obj_id)

    # add info to adata
    co_occ = np.array(co_occs)
    for i,c1 in enumerate(self.clusters):
        for j,c2 in enumerate(self.clusters):
            df = pd.DataFrame(co_occ[:,i,j], index=obj_ids, columns=range(len(interval)-1))
            df.index = df.index.astype(str)
            # ensure obj_ids are in correct order
            df = pd.merge(df, self.adata.obs, how='right', left_index=True, right_on='mapobject_id', suffixes=('','right'))[df.columns]
            # add to adata.obsm
            self.adata.obsm[f'co_occurence_{c1}_{c2}'] = df

    self.adata.uns['co_occurence_params'] = {'interval': list(interval)}
    self.log.info(f'saving adata to {self.fname}')
    self.adata.write(self.fname)


interval = np.linspace(0,80,3).astype(np.float32)
co_occs, obj_ids = extract_co_occurrence(extr, interval)

def co_occurrence(adata, interval, cluster_id='cluster_id'):
    """
    co occurrence
    returns co_occ, mapobject_ids, well_name
    """
    # calculate neighborhood enrichment for each cell individually
    co_occs = []
    mapobject_ids = []
    well_name = []
    
    log.info(f'calculating co-occurrence for intervals {interval} and clustering {cluster_id}')
    num_clusters = len(adata.obs[cluster_id].cat.categories)
    cluster_names = {n: i for i,n in enumerate(adata.obs[cluster_id].cat.categories)}
    for mapobject_id in np.unique(adata.obs.mapobject_id)[:5]:
        adata_cell = adata[adata.obs.mapobject_id == mapobject_id].copy()
        log.info(f'co-occurrence for {mapobject_id}, with shape {adata_cell.shape}')
        cur_co_occ, cur_interval = sq.gr.co_occurrence(
            adata_cell,
            cluster_key=cluster_id,
            spatial_key='spatial',
            interval=interval,
            copy=True, show_progress_bar=False,
        )
        # ensure that co_occ always has same format 
        co_occ = np.zeros((num_clusters,num_clusters,len(interval)-1))
        cur_clusters = np.vectorize(cluster_names.__getitem__)(np.array(adata_cell.obs[cluster_id].cat.categories))
        grid = np.meshgrid(cur_clusters, cur_clusters)
        co_occ[grid[0].flat, grid[1].flat] = cur_co_occ.reshape(-1, len(interval)-1)
        co_occs.append(co_occ.copy())

        mapobject_ids.append(mapobject_id)
        well_name.append(adata_cell.obs['well_name'][0])
        
    comb_co_occs = np.array(co_occs)
    # replace nan with 0 and inf with max value
    max_co_occ = np.nan_to_num(comb_co_occs, posinf=0).max()
    comb_co_occs = np.nan_to_num(comb_co_occs, posinf=max_co_occ)
    
    return comb_co_occs, mapobject_ids, well_name


INFO:FeatureExtractor:calculating co-occurrence for intervals [ 0. 40. 80.] and clustering clustering_0.5 (col: clustering_0.5)
INFO:MPPData:Before subsetting: 557 objects
INFO:MPPData:Subsetting to 1 objects
INFO:MPPData:Created new: MPPData for NascentRNA (17008 mpps with shape (1, 1, 34) from 1 objects). Data keys: ['x', 'y', 'obj_ids', 'mpp', 'labels', 'clustering_0.5', 'latent', 'conditions'].
INFO:FeatureExtractor:co-occurrence for 199632, with shape (17008, 34)
INFO:MPPData:Before subsetting: 557 objects
INFO:MPPData:Subsetting to 1 objects
INFO:MPPData:Created new: MPPData for NascentRNA (6936 mpps with shape (1, 1, 34) from 1 objects). Data keys: ['x', 'y', 'obj_ids', 'mpp', 'labels', 'clustering_0.5', 'latent', 'conditions'].
INFO:FeatureExtractor:co-occurrence for 199634, with shape (6936, 34)
INFO:MPPData:Before subsetting: 557 objects
INFO:MPPData:Subsetting to 1 objects
INFO:MPPData:Created new: MPPData for NascentRNA (11337 mpps with shape (1, 1, 34) from 1 objects). Dat

TypeError: 0 of type <class 'int'> is an invalid key. Should be str.

Above error raised while writing key 'obsm/co_occurence_0_0' of <class 'h5py._hl.files.File'> from /.

In [55]:
co_occ = np.array(co_occs)

In [56]:
co_occ.shape

(10, 9, 9, 2)

In [62]:
import pandas as pd

In [71]:
self = extr
for i,c1 in enumerate(self.clusters):
    for j,c2 in enumerate(self.clusters):
        df = pd.DataFrame(co_occ[:,i,j], index=obj_ids, columns=range(len(interval)-1))
        df.index = df.index.astype(str)
        # ensure obj_ids are in correct order
        df = pd.merge(df, self.adata.obs, how='right', left_index=True, right_on='mapobject_id', suffixes=('','right'))[df.columns]
        # add to adata.obsm
        self.adata.obsm[f'co_occurence_{c1}_{c2}'] = df
        break
    break

In [74]:
self.adata.obsm['co_occurence_0_0']

,0,1
0,1.145597,0.902322
1,1.361278,0.656402
2,1.506623,0.728965
3,1.337002,0.762019
4,1.253471,0.902481
...,...,...
552,NaN,NaN
553,NaN,NaN
554,NaN,NaN
555,NaN,NaN


In [38]:
adata.obs['clustering_0.5'].cat.categories

Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object')

In [3]:
extr.extract_intensity_size()

INFO:FeatureExtractor:Calculating clustering_0.5 (col: clustering_0.5) mean and size for 184A1_unperturbed/I09
INFO:MPPData:Created new: MPPData for NascentRNA (12132995 mpps with shape (1, 1, 35) from 886 objects). Data keys: ['x', 'y', 'obj_ids', 'mpp', 'labels'].
INFO:MPPData:Loaded data from 184A1_unperturbed/I09.
INFO:MPPData:Setting mpp to empty array
INFO:MPPData:Created new: MPPData for NascentRNA (7218204 mpps with shape (1, 1, 34) from 557 objects). Data keys: ['y', 'obj_ids', 'x', 'clustering_0.5', 'latent', 'mpp'].
INFO:MPPData:Before subsetting: 886 objects
INFO:MPPData:Subsetting to 557 objects
INFO:MPPData:Updated data to keys ['x', 'y', 'obj_ids', 'mpp', 'labels', 'clustering_0.5', 'latent']
INFO:MPPData:Loaded data from 184A1_unperturbed/I09, with base data from 184A1_unperturbed/I09
INFO:MPPData:Restricted channels to 34 channels
INFO:MPPData:Subtracting channel-specific background value defined in column mean_background
INFO:MPPData:Rescaling MPP intensities per chan

## Explore extracted features

In [7]:
extrs = [FeatureExtractor.from_adata(os.path.join(exp.full_path, 'aggregated/full_data', data_dir, 'features_seed1.h5ad')) for data_dir in exp.data_params['data_dirs']]

# get combined adata for dotplots
adatas = [extr.get_intensity_adata() for extr in extrs]
adata = ad.concat(adatas, index_unique='-')
zscore(adata, limit_to_groups={'perturbation_duration':'unperturbed'})

In [5]:
plot_mean_intensity(adata, groupby='cluster', limit_to_groups={'perturbation_duration':'unperturbed'}, dendrogram=True, layer='zscored', cmap='bwr', vmin=-10, vmax=10)
plot_mean_size(adata, groupby_row='cluster', groupby_col='perturbation_duration', normby_row='all', vmax=0.3)

In [ ]:
res = get_intensity_change(adata, groupby='cluster', reference_group='perturbation_duration', reference='unperturbed', color='logfoldchange', size='pval')
plot_intensity_change(**res, adjust_height=True, figsize=(15,5), vmin=-2, vmax=2, dendrogram=True)

res = get_intensity_change(adata, groupby='cluster', reference_group='perturbation_duration', reference='unperturbed', color='logfoldchange', size='pval', norm_by_group='all')
plot_intensity_change(**res, adjust_height=True, figsize=(15,5), vmin=-2, vmax=2)